In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import numpy as np

In [2]:
Actions = np.array([np.array([0, 1, 0]),
np.array([0, 0, 0]),
np.array([0, 0, 0.4]),
np.array([-1, 0, 0]),
np.array([1, 0, 0]),
np.array([-1, 0, 0.4]),
np.array([1, 0, 0.4])])

In [3]:
model = load_model('my_model.h5')

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5, 96, 96, 3)]    0         
_________________________________________________________________
conv3d (Conv3D)              (None, 5, 96, 96, 32)     2432      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 48, 48, 32)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 48, 48, 64)     51264     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 24, 24, 64)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 5, 24, 24, 128)    204928    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 5, 12, 12, 128)    0     

In [5]:
import gym
env = gym.make('CarRacing-v0')
state = env.reset()
states = np.empty((1,5,96,96,3))
for i in range(5):
    states[:,i] = state
    
for _ in range(50):
    action = np.array([0,1,0])
    next_state, reward, done, _ = env.step(action)
    env.render()
    states = np.append(next_state.reshape(1,1,96,96,3), states[:,:-1], axis=1)
for _ in range(3000):
    action_dist = model.predict(states)
    action_num = np.argmax(action_dist)
    action = Actions[action_num]
    next_state, reward, done, _ = env.step(action)
    env.render()
    states = np.append(next_state.reshape(1,1,96,96,3), states[:,:-1], axis=1)
    if done:
        break
     # take a random action
env.close()

Track generation: 1067..1345 -> 278-tiles track


D:\Anaconda\envs\CarRace\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
